In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from numpy import exp
from scipy import ndimage
from func import *
from astropy.table import Table, hstack
import os
import glob
from astropy.table import Table, vstack, join
import pandas as pd
from shutil import copyfile

/home/mainak/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/mainak/.local/lib/python2.7/site-packages/pyneb/utils/manage_atomic_data.py:528: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  gsconf = np.genfromtxt(execution_path('../atomic_data/levels/gsconfs.dat'), names=['atom', 'gsconf'], dtype=None)


In [2]:
def moffat_fit(data,error,box_size,amp,x0,y0,gamma_fixed=None,alpha_fixed=None,MC_loops=100):
    y, x = np.mgrid[:box_size, :box_size] 
    if gamma_fixed is None and alpha_fixed is None:
        p_init = models.Moffat2D(amplitude=amp,x_0=x0,y_0=y0,gamma=2,alpha=2)
    else:
        p_init = models.Moffat2D(amp,x0,y0,gamma_fixed,alpha_fixed,fixed={'gamma':True,'alpha':True})
    f = fitting.LevMarLSQFitter()
    p = f(p_init, x, y, data)
    model = p(x,y)
    residual = data - model
    res = (residual/error)
    [amp_out,x0_out,y0_out,gamma_out,alpha_out]= p.parameters
    fwhm_out = 2*gamma_out*np.sqrt(2**(1/alpha_out)-1)
    p_parameters = np.append(p.parameters,fwhm_out)
    parameters_MC = np.zeros((len(p_parameters),MC_loops))
    for l in range(MC_loops):
        iteration_data = np.random.normal(data,error) 
        if gamma_fixed is None and alpha_fixed is None:
            p_MC_init = models.Moffat2D(amplitude=amp,x_0=x0,y_0=y0,gamma=2,alpha=2)
        else:
            p_MC_init = models.Moffat2D(amp,x0,y0,gamma_fixed,alpha_fixed,fixed={'gamma':True,'alpha':True})
        f = fitting.LevMarLSQFitter()
        p_MC = f(p_MC_init, x, y, iteration_data)
        [amp_MC,x0_MC,y0_MC,gamma_MC,alpha_MC]= p_MC.parameters
        fwhm_MC = 2*gamma_MC*np.sqrt(2**(1/alpha_MC)-1)
        p_MC_parameters = np.append(p_MC.parameters, fwhm_MC)
        parameters_MC[:,l] = p_MC_parameters    
    parameters_err = np.std(parameters_MC,1) 
    [amp_err,x0_err,y0_err,gamma_err,alpha_err,fwhm_err] = parameters_err    
    if gamma_fixed is None and alpha_fixed is None:
        (par,err) = ([amp_out,x0_out,y0_out,gamma_out,alpha_out,fwhm_out],[amp_err,x0_err,y0_err,gamma_err,alpha_err,fwhm_err])   
    else:
        (par,err) = ([amp_out,x0_out,y0_out],[amp_err,x0_err,y0_err])   
    return par,err,model,res



In [3]:
def maps(Hb_blr_br_data,OIII_br_data,OIII_nr_data,Hb_model,OIII_br_model,OIII_nr_model,Hb_dev,OIII_br_dev,OIII_nr_dev,obj,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    hdus=[]
    hdus.append(fits.PrimaryHDU())
    hdus.append(fits.ImageHDU(Hb_blr_br_data,name='Hb_blr_br_data'))
    hdus.append(fits.ImageHDU(OIII_br_data,name='OIII_br_data'))
    hdus.append(fits.ImageHDU(OIII_nr_data,name='OIII_nr_data'))
    hdus.append(fits.ImageHDU(Hb_model,name='Hb_blr_br_model'))
    hdus.append(fits.ImageHDU(OIII_br_model,name='OIII_br_model'))   
    hdus.append(fits.ImageHDU(OIII_nr_model,name='OIII_nr_model'))
    hdus.append(fits.ImageHDU(Hb_dev,name='Hb_blr_br_res'))
    hdus.append(fits.ImageHDU(OIII_br_dev,name='OIII_br_res'))
    hdus.append(fits.ImageHDU(OIII_nr_dev,name='OIII_nr_res'))
    hdu = fits.HDUList(hdus)
    hdu.writeto('%s/%s/maps_flux_mod_%s.fits'%(destination_path_cube,obj,obj),overwrite='True')
    
def moffat_table(full_data,full_error,obj,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    column_names={'amp_Hb_blr':0,'x0_Hb_Blr':1,'y0_Hb_Blr':2,'gamma':3,'alpha':4,'fwhm':5,'amp_OIII_br':6,'x0_OIII_br':7,'y0_OIII_br':8,'amp_OIII_nr':9,'x0_OIII_nr':10,'y0_OIII_nr':11}
    columns=[]
    for key in column_names.keys():
        columns.append(fits.Column(name=key,format='E',array=[full_data[column_names[key]]]))
        columns.append(fits.Column(name=key+'_err',format='E',array=[full_error[column_names[key]]]))
    coldefs = fits.ColDefs(columns)
    hdu = fits.BinTableHDU.from_columns(coldefs)
    hdu.writeto('%s/%s/moffat_table_flux_mod_%s.fits'%(destination_path_cube,obj,obj),overwrite=True)
    
def flux_compare(obj,Hb_blr_flux,OIII_br_data,Hb_model,OIII_br_model,Hb_blr_flux_err,OIII_br_err,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    f_blr_data = np.sum(Hb_blr_flux)
    f_wing_data = np.sum(OIII_br_data)
    f_blr_model = np.sum(Hb_model)
    f_wing_model = np.sum(OIII_br_model)
    print f_blr_data,f_blr_model
    f_blr_err = np.sqrt(np.sum(Hb_blr_flux_err**2))
    f_wing_err = np.sqrt(np.sum(OIII_br_err**2))
    tab_par = [f_blr_data,f_wing_data,f_blr_model,f_wing_model]
    tab_err = [f_blr_err,f_wing_err,0,0]
    column_names={'flux_blr_data':0,'flux_wing_data':1,'flux_blr_model':2,'flux_wing_model':3}
    columns=[]
    for key in column_names.keys():
        columns.append(fits.Column(name=key,format='E',array=[tab_par[column_names[key]]]))
        columns.append(fits.Column(name=key+'_err',format='E',array=[tab_err[column_names[key]]]))
    coldefs = fits.ColDefs(columns)
    hdu = fits.BinTableHDU.from_columns(coldefs)
    hdu.writeto('%s/%s/%s_flux_HbO3.fits'%(destination_path_cube,obj,obj),overwrite=True)
    
def flux_comp_table(obj,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    t1 = Table.read('%s/%s/source_%s.fits'%(destination_path_cube,obj,obj),format='fits')
    t2 = Table.read('%s/%s/%s_flux_HbO3.fits'%(destination_path_cube,obj,obj),format='fits')
    new = hstack([t1, t2])
    new.write('%s/%s/%s_flux_HbO3.fits'%(destination_path_cube,obj,obj),overwrite=True)

In [4]:
def flux_data_err(obj,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    hdu = fits.open('%s/Flux Maps/%s/subcube_par_%s.fits'%(destination_path_cube,obj,obj))
    (OIII_nr,OIII_br,Hb1_blr_br,Hb2_blr_br) = (hdu[2].data,hdu[3].data,hdu[5].data,hdu[6].data)
    hdu.close()
    
    hdu = fits.open('%s/Flux Maps/%s/subcube_par_err_%s.fits'%(destination_path_cube,obj,obj))
    (OIII_nr_err,OIII_br_err,Hb1_blr_br_err,Hb2_blr_br_err) = (hdu[2].data,hdu[3].data,hdu[5].data,hdu[6].data)
    hdu.close()
    
    hdu = fits.open('%s/%s/%s_empirical_factor_subcube.fits'%(destination_path_cube,obj,obj))
    central_tab = hdu[1].data
    central_columns = hdu[1].header
    (emp_Hb1,emp_Hb2,emp_wing) = (central_tab.field('emp_fact_Hb1')[0],central_tab.field('emp_fact_Hb2')[0],central_tab.field('emp_fact_wing')[0])
    hdu.close()
    
    (amp_OIII_nr,amp_OIII_br,amp_Hb1_blr_br,amp_Hb2_blr_br) = (np.max(OIII_nr),np.max(OIII_br),np.max(Hb1_blr_br),np.max(Hb2_blr_br))
    if amp_Hb1_blr_br > amp_Hb2_blr_br:
        (Hb_blr_br,amp_Hb_blr_br,Hb_blr_err,emp_Hb_blr) = (Hb1_blr_br,amp_Hb1_blr_br,Hb1_blr_br_err,emp_Hb1)
    else:
        (Hb_blr_br,amp_Hb_blr_br,Hb_blr_err,emp_Hb_blr) = (Hb2_blr_br,amp_Hb2_blr_br,Hb2_blr_br_err,emp_Hb2)
    
    (blr_err_final,wing_err_final,core_err_final) = (emp_Hb_blr*Hb_blr_err,emp_wing*OIII_br_err,OIII_nr_err)
    return Hb_blr_br,OIII_br,OIII_nr,amp_Hb_blr_br,amp_OIII_br,amp_OIII_nr,blr_err_final,wing_err_final,core_err_final
    

In [5]:
def algorithm_script(obj,z,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):  
    
    Hb_blr_flux = loadblr_flux('%s/Flux Maps/%s/fluxcube_blr_%s.fits'%(destination_path_cube,obj,obj))
    Hb_blr_flux_err = loadblr_flux('%s/Flux Maps/%s/fluxcube_blr_err_%s.fits'%(destination_path_cube,obj,obj))
    [y0_blr_flux,x0_blr_flux] = ndimage.measurements.maximum_position(Hb_blr_flux)
    blr_amp = np.max(Hb_blr_flux)
    #print np.sum(Hb_blr_flux )
    
    (Hb_blr_br_data,OIII_br_data,OIII_nr_data,amp_Hb_blr_br,amp_OIII_br,amp_OIII_nr,Hb_blr_br_err,OIII_br_err,OIII_nr_err) = flux_data_err(obj)
    #print Hb_blr_br_data#,Hb_blr_br_err
    box_size = np.shape(Hb_blr_flux)[1]
    #print box_size   
    
    (brightest_pixel_Hb_blr_br_x,brightest_pixel_Hb_blr_br_y,brightest_pixel_OIII_br_x,brightest_pixel_OIII_br_y,brightest_pixel_OIII_nr_x,brightest_pixel_OIII_nr_y) = brightest_pixel_flux_map(Hb_blr_br_data,OIII_br_data,OIII_nr_data) 
    
    (Hb_par,Hb_error,Hb_model,Hb_res) = moffat_fit(Hb_blr_flux,Hb_blr_flux_err,box_size,blr_amp,x0_blr_flux,y0_blr_flux,None,None,100)
    #print np.sum(Hb_model)
    (gamma_fix,alpha_fix) = (Hb_par[3],Hb_par[4])#these two are gamma and alpha
    #print Hb_par
    (OIII_br_par,OIII_br_error,OIII_br_model,OIII_br_res) = moffat_fit(OIII_br_data,OIII_br_err,box_size,amp_OIII_br,brightest_pixel_OIII_br_x,brightest_pixel_OIII_br_y,gamma_fix,alpha_fix,100)   

    (OIII_nr_par,OIII_nr_error,OIII_nr_model,OIII_nr_res) = moffat_fit(OIII_nr_data,OIII_nr_err,box_size,amp_OIII_nr,brightest_pixel_OIII_nr_x,brightest_pixel_OIII_nr_y,gamma_fix,alpha_fix,100)

    maps(Hb_blr_br_data,OIII_br_data,OIII_nr_data,Hb_model,OIII_br_model,OIII_nr_model,Hb_res,OIII_br_res,OIII_nr_res,obj)
    
    (full_data,full_error) = (np.append(Hb_par,[OIII_br_par,OIII_nr_par]),np.append(Hb_error,[OIII_br_error,OIII_nr_error]))
    
    moffat_table(full_data,full_error,obj)  
    
    flux_compare(obj,Hb_blr_flux,OIII_br_data,Hb_model,OIII_br_model,Hb_blr_flux_err,OIII_br_err)
    
    flux_comp_table(obj)
    
    print '%s.fits'%(obj)


In [6]:
z = {"HE0021-1810":0.05352}
objs = z.keys()


for obj in objs:
    algorithm_script(obj,z[obj])

133.87207 131.93387941743953
HE0021-1810.fits


In [7]:
z = {"HE0021-1810":0.05352,"HE0021-1819":0.053197,"HE0040-1105":0.041692,"HE0108-4743":0.02392,"HE0114-0015":0.04560
    ,"HE0119-0118":0.054341,"HE0212-0059":0.026385,"HE0224-2834":0.059800,"HE0227-0913":0.016451,"HE0232-0900":0.043143
    ,"HE0253-1641":0.031588,"HE0345+0056":0.031,"HE0351+0240":0.036,"HE0412-0803":0.038160,"HE0429-0247":0.042009
    ,"HE0433-1028":0.035550,"HE0853+0102":0.052,"HE0934+0119":0.050338,"HE1011-0403":0.058314,"HE1017-0305":0.049986
    ,"HE1029-1831":0.040261,"HE1107-0813":0.058,"HE1108-2813":0.024013,"HE1126-0407":0.061960,"HE1237-0504":0.009
    ,"HE1248-1356":0.01465,"HE1330-1013":0.022145,"HE1353-1917":0.035021,"HE1417-0909":0.044,"HE2128-0221":0.05248
    ,"HE2211-3903":0.039714,"HE2222-0026":0.059114,"HE2233+0124":0.056482,"HE2302-0857":0.046860}

objs = z.keys()


for obj in objs:
    algorithm_script(obj,z[obj])

9275.441 9233.922748787838
HE0345+0056.fits
2939.684 2912.7909583915225
HE0412-0803.fits
701.65955 701.1541132269556
HE0224-2834.fits
3362.5732 3320.035949164978
HE0433-1028.fits
401.86118 398.6884824950925
HE2233+0124.fits
436.70734 436.096741145751
HE1330-1013.fits
1260.9509 1250.328094567778
HE1011-0403.fits
1319.459 1311.33842905205
HE0429-0247.fits
133.87207 131.93387941743953
HE0021-1810.fits
4314.687 4308.419168142756
HE0227-0913.fits
721.3879 720.2532800003551
HE1029-1831.fits
9506.619 9504.288444681097
HE0232-0900.fits
334.57407 337.7218993679302
HE2222-0026.fits
2837.1924 2804.417596367748
HE2302-0857.fits
87.668816 87.86277953286105
HE0021-1819.fits
191.73672 190.02720667742196
HE2128-0221.fits
169.60056 166.341291741088
HE0853+0102.fits
2599.8877 2561.2593802175784
HE1237-0504.fits
2146.0415 2129.3670918377593
HE1107-0813.fits
85.23442 83.38009177134488
HE0114-0015.fits
479.32794 470.0072681399864
HE2211-3903.fits
1578.0405 1565.1307196010853
HE0351+0240.fits
366.19885 363.